# Transport for London (TfL) Journeys - Exploratory Data Analysis

This is an exploratory data analysis project. The aim of this project is to identify the main characteristics of a dataset, extract information without using a formal model and display insights visually in a way that would be understood by stakeholders.

I chose this dataset to analyse, because I am a regular user of the Transport for London (TfL) network. I find myself constantly trying to avoid travelling during busier times or at busier stations. I thought that it would be interesting to analyse the data from the point of view of a TfL employee who is trying to reduce overcrowded stations.

## The Dataset

https://www.kaggle.com/datasets/astronasko/transport-for-london-journey-information

The dataset contains data on Oyster Card journeys from a week in November of 2009. Oyster cards are reusable cards used for ticketing purposes on the TfL network, so they can be used on buses, underground trains, tram lines etc.

The features in the dataset are either about the journey taken (e.g. Start/Exit station, Route numbers) or the Oyster Card type. 

### Limitations of the Dataset 

- All the data is from a single week in November 2009, that may not be representative of all weeks.
- Since the data is from over 15 years ago, it may inaccurately describe what journeys would look like today. Many things have changed such as new service lines, different payment methods. Because of this, I want to ensure that any analysis applied on this dataset could be applied on a dataset from 2024 as well.

## Analysis

Questions to answer in the initial analysis:
- How many journeys were made for each mode of transport (i.e. bus, DLR, underground etc.)?
- What is the distribution of journey start times? This can help to identify the busiest times.
- Which stations are used the most?
- Which products are used to pay for journeys the most/least?

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import time
import math
import folium

In [ ]:
df = pd.read_csv("Nov09JnyExport.csv")
df.head()

In [ ]:
row_count = len(df)
df.info(show_counts=True)

The data has 2623487 rows. The summary above shows that there are no Null values.

I'm going to find out which mode of transport is most popular.

In [ ]:
df['SubSystem'].value_counts()

The top three modes of transport London Buses (LTB), London Underground (LUL) and National Rail (NL) make up 97% of all journeys.

What is the distribution of journey start times?

In [ ]:
df["EntTimeHoursAfterMidnight"] = df["EntTime"]/60
ax = sns.displot(df, x="EntTimeHoursAfterMidnight")
ax.set(xlabel="Journey entry time (hours after midnight)", ylabel='Number of journeys', title="Graph showing distribution of journey start times")
plt.show

From the graph, we can see that the data collection starts just before 5am on the first day and stops at around the same time the next day.

It seems that the peak times for journeys started are around 8am and between 3pm and 6pm.
Between these times, there are also a considerable amount of journies. Between 1am and 5am there are very little journeys started.

What about the sharp spike of data at 0 (midnight)? It's unlikely that data collection was occuring at this time, as most of the data starts at 5am. Even if data was being collected, I would not expect over 40,000 journeys to have started at this time, as the number is at around 2000 the next day.

I expect this has to do with 'unstarted' journeys (where the customer may have forgotten to 'tap' their Oyster card on entering the TfL Network). I'm going to verify this.

In [ ]:
row_count = len(df)
ent_time_0_count = len(df[df.EntTime == 0])
unstarted_journey_count = len(df[df.StartStn == "Unstarted"])
print("Number of journies that started at midnight: ", ent_time_0_count)
print("Number of 'unstarted' journies: ", unstarted_journey_count)
print("\'Unstarted\' journies make up {}% of all journies ".format(unstarted_journey_count*100/row_count))

The numbers seem to confirm that the spike of journeys 'started' at midnight correspond to the number of unstarted journeys.
Since these journies make up less than 2% of all journies, it would be safe to drop these instances if needed.

Next, I want to find out which stations are the busiest. This will be informative when trying to reduce overcrowding on the network.

In [ ]:
# Combine the start and end stations into a series.
# Ignoring bus journeys as their stations are just recorded as 'Bus'
start_stations = df[~df.StartStn.isin(["Unstarted", "Bus"])]["StartStn"]
end_stations = df[~df.EndStation.isin(["Unfinished", "Bus"])]["EndStation"]
all_stations = start_stations._append(end_stations, ignore_index=True)

# Count the frequency of each station
all_stations_count = all_stations.value_counts()

all_stations_count

There are 453 stations recorded (note: some stations with multiple entrances/exits are recorded as separate stations). There is a large difference in counts between the most and least popular stations.

Next, I'm going to plot the frequencies for the top 7 stations.

In [ ]:
stations_highest_count = all_stations_count.nlargest(7)
stations_highest_count_df = stations_highest_count.reset_index()
stations_highest_count_df.columns = ['Station', 'Frequency']

# Plot the data
plt.figure(figsize=(10, 6))
sns.barplot(x='Station', y='Frequency', data=stations_highest_count_df)
plt.title('Top 7 most used stations')
plt.xlabel('Station')
plt.ylabel('Frequency')
plt.show()

## Solving the Problem - Overcrowded Stations

I found a list of station and their coordinates provided by TfL here: https://tfl.gov.uk/corporate/transparency/freedom-of-information/foi-request-detail?referenceId=FOI-1451-1819

I will use this to show the most popular stations and quieter surrounding stations.

In [ ]:
station_df = pd.read_csv("station_coordinates.csv", index_col=[0])
station_df.head()

In [ ]:
# Some stations e.g. Liverpool Street have more than one row (as they have multiple entrances)
station_df = station_df.drop_duplicates(subset='NAME', keep='first')

In [ ]:
all_stations_count = all_stations_count.reset_index()
all_stations_count.columns = ["Station", "Frequency"]
all_stations_count.to_csv("all_Stations")

In [ ]:
all_kings_x_stations = all_stations_count[all_stations_count['Station'].str.startswith('Kings Cross')]
kings_cross_total_frequency = all_kings_x_stations.Frequency.sum()

In [ ]:
# In the original data frame, Liverpool Street and Kings Cross have different names.
# Update them to match station_df

# Adding the new row using append
all_stations_count.loc[len(all_stations_count), all_stations_count.columns] = "King's Cross St. Pancras", kings_cross_total_frequency
all_stations_count = all_stations_count[~all_stations_count['Station'].isin(all_kings_x_stations['Station'])]

In [ ]:
def get_journey_count(station_name):
    if station_name in all_stations_count["Station"].values:
        return all_stations_count.loc[all_stations_count["Station"] == station_name]["Frequency"].iloc[0]
    return 0

station_df["journey_count"] = station_df["NAME"].apply(lambda x: get_journey_count(x))
station_df.head()

In [ ]:
m = folium.Map(location=[51.51398203, -0.0742363], tiles='OpenStreetMap', zoom_start=12)
busy_stations = ["Oxford Circus", "Victoria", "London Bridge", "Liverpool Street", "Canary Wharf",
                "Stratford", "King's Cross St. Pancras"]

for i in range(0, len(station_df)):
    station_name = station_df.iloc[i]['NAME']
    station_zone = station_df.iloc[i]['Zone']
    color = 'blue'
    if station_name in busy_stations:
        color='red'
    if station_zone in [1, 2, 3]:
        folium.Circle(
            location = [station_df.iloc[i]['y'], station_df.iloc[i]['x']],
            popup= station_name,
            radius = float(station_df.iloc[i]["journey_count"])/100,
            color=color,
            fill=True,
            fill_color=color
        ).add_to(m)
m

This map shows stations in Zones 1 to 3.
The red dots are the busiest stations. 
The radius of the dots show how busy the stations are. 
This can be used to direct customers to nearby, less busy station_coordinates.csv
E.g. Customers who usually go to Liverpool Street Station could instead use Moorgate or Aldgate.